# Network Analysis - Generate a interest graph using the GitHub API
Bom, nesse artigo iremos adentrar no estudos de análises em rede utilizando python, em especial gerar um grafo de interesse para extrair algumas análises. No nosso caso de estudos usaremos a API do GitHub e um repositório de código aberto, selecionaremos quem o "segue" e veremos se esses usuários possuem relação entre si. Here we go...

## 1 Obter acesso à API do GitHub
Existem duas formas de obter esse acesso, criando um repositório responsável por acessar a api ou gerando um token pessoal para isso. Vamos optar pela segunda opção. Ainda, temos a opção de gerar esse token via interface de usuário ou programaticamente. Vamos utilizar o método programático (e mais legal, off course).


In [0]:
# instalando a lib requests
!pip install requests

In [0]:
# obtendo token de acesso programaticamente
import requests
import json

# credenciais da conta
username = '' # you github username
password = '' # you github password

url = 'https://api.github.com/authorizations'
note = 'Mining repositories to network analysis'
post_data = {'scopes': ['repo'], 'note': note}

response  = requests.get(url,
                        auth = (username, password),
                        data = json.dumps(post_data)
                       )

print("API response: ", response.text)
print("Your OAuth token is: ", response.json()[0]['hashed_token'])

API response:  [{"id":260105660,"url":"https://api.github.com/authorizations/260105660","app":{"name":"git: https://github.com/ on M40893 at 11-fev-2019 12:25","url":"https://developer.github.com/v3/oauth_authorizations/","client_id":"00000000000000000000"},"token":"","hashed_token":"20939728e30431899de75d8543062af879334a0a051e5c7e7778ebb571fcb000","token_last_eight":"6e0fbb26","note":"git: https://github.com/ on M40893 at 11-fev-2019 12:25","note_url":null,"created_at":"2019-02-11T15:25:59Z","updated_at":"2019-02-11T15:25:59Z","scopes":["gist","repo"],"fingerprint":null},{"id":301738068,"url":"https://api.github.com/authorizations/301738068","app":{"name":"Mining repositories to network analysis","url":"https://developer.github.com/v3/oauth_authorizations/","client_id":"00000000000000000000"},"token":"","hashed_token":"c33390a4e22bce805aac846611f6bed3a14295203885eaccfeb3a7e367052f63","token_last_eight":"8a1c821e","note":"Mining repositories to network analysis","note_url":null,"create

## 2 Gerando grafo de stargazers 
Vamos gerar um grafo dos "stargazers" (vulgo perseguidores de estrelas (?)). Iremos utilizar alguma libs python aqui para nos auxiliar no processo. Para a manipulação de grafos usáremos o [Networkx](https://networkx.github.io/) e a [PyGithub](https://pygithub.readthedocs.io/en/latest/) para facilitar os acessos a api do github. As outras depedências que forem necessárias seram citadas no decorrer do texto.

### 2.1 Obtendo stargazers
Como em geral não a biblioteca PyGithub não vem instalada, precisamos instalar com o comando:


In [1]:
# instalar PyGithub
!pip install PyGithub

     |████████████████████████████████| 112kB 4.2MB/s 
  Stored in directory: /root/.cache/pip/wheels/41/a2/13/f40c7a91db4b3743ef1830234526054d62f8c502510f3eb18a
Successfully built PyGithub


Precisamo escolher um repositório. Escolhi o [Mining-the-Social-Web ](https://github.com/ptwobrussell/Mining-the-Social-Web) do [ptwobrussell](https://github.com/ptwobrussell) para realizar nosso estudo, pois é um repositório com um número não muito grande de "seguidores", facilitando na exibição do grafo. Mas fique a vontade para escolher qualquer um de sua preferência. Na *seção 1* obtemos um token de acesso para api do github. É aqui que precisaremos dela.

In [13]:
# import ygithub
from github import Github

# especificar meu token de acesso
ACCESS_TOKEN = ''

# especificando um usuário e um repositório de interesse
USER = 'ptwobrussell'
REPO = 'Mining-the-Social-Web'

# autenticando na api do github
client = Github(ACCESS_TOKEN, per_page=100)

# especificando usuário que estamos buscando: ptwobrussell
user = client.get_user(USER)

# acessando o nosso repositório: Mining-the-Social-Web 
repo = user.get_repo(REPO)

# obtendo stargazers do repositório
stargazers = [s for s in repo.get_stargazers()]
print("Number of stargazers", len(stargazers))

# lista de stargazers
print(stargazers)


Number of stargazers 1182
[NamedUser(login="rdempsey"), NamedUser(login="frac"), NamedUser(login="prb"), NamedUser(login="mcroydon"), NamedUser(login="batasrki"), NamedUser(login="twleung"), NamedUser(login="kevinchiu"), NamedUser(login="nikolay"), NamedUser(login="tswicegood"), NamedUser(login="ngpestelos"), NamedUser(login="darron"), NamedUser(login="brunojm"), NamedUser(login="rgaidot"), NamedUser(login="openweb"), NamedUser(login="shanlalit"), NamedUser(login="hoffmann"), NamedUser(login="nacht"), NamedUser(login="hectoregm"), NamedUser(login="tzuryby"), NamedUser(login="marksands"), NamedUser(login="wbzyl"), NamedUser(login="sou"), NamedUser(login="magnum"), NamedUser(login="suzuki"), NamedUser(login="tertsch"), NamedUser(login="ymirpl"), NamedUser(login="sebasmagri"), NamedUser(login="galvez"), NamedUser(login="paulbersch"), NamedUser(login="georgebellos"), NamedUser(login="acadopia"), NamedUser(login="ggtr1138"), NamedUser(login="program247365"), NamedUser(login="edwelker"), Nam

### 2.2 Networkx
Agora podemos criar nosso grafo utilizando como base nosso repositório e seus stargazers. Vamos fazer aqui uma distinção entre repositório e usuário pois, é bastante comum existir repositórios com o mesmo nome de usuários. Para isso vamos adicionar o pós-fixo "(user)" para usuário e "(repo)" para repositório. Aqui vamos utilizar o Networkx, sua utilização é simples, porém bastante poderosa. Vamos criar nosso grafo...

In [14]:
import networkx as nx

# cria novo grafo direcionado (DiGraph)
graph = nx.DiGraph()
graph.name = "Interest graph of a repo"

# adicionando o node repositório 'requests'
graph.add_node(repo.name + '(repo)', type='repo', lang=repo.language, owner=user.login)

# adicionando node de stargazers e edges com repositório
for stargazer in stargazers:
  graph.add_node(stargazer.login + '(user)', type="user")
  graph.add_edge(stargazer.login + '(user)', repo.name + '(repo)', type="gazes")
  
# obtendo informações do grafo
print(nx.info(graph))

Name: Interest graph of a repo
Type: DiGraph
Number of nodes: 1183
Number of edges: 1182
Average in degree:   0.9992
Average out degree:   0.9992


Vemos que nosso grafo tem um número não muito grande de nós: 1183. Com nosso grafo criado, podemos fazer nossa primeira visualização. É importante ressaltar que o Networkx não foi desenvolvido para trabalhar com visualização de grafos, apenas com sua manipulação. Então, iremos utilizar um software externo para gerar uma visualização mais amigável. Não vou entrar em detalhes sobre o gephi, pois não é esse o foco do artigo. Qualquer dúvida, veja a [documentação](https://gephi.org/users/) da ferramenta 

In [0]:
!pip install git+https://github.com/ericmjl/nxviz

In [0]:
import matplotlib.pyplot as plt
from nxviz import CircosPlot

c = CircosPlot(graph, 
               node_color='type',
               node_order='type',
               figsize=(30, 30),
               nodeprops={"radius": 1},
               fontsize = 12
              )

c.draw()
c.figure.tight_layout()

plt.show()

Isso pode levar algum tempo, devido a quantidade de nós no grafo. Mas temos uma alternativa também, até visualmente melhor: o Gephi. O networkx permite exportar o grafo com a extensão .graphml e com isso é possível realizar a leitura no Gephi.

In [0]:
nx.write_graphml(graph, "first.graphml")

### 2.3 Adicionando seguidores ao ao grafo
Para reduzir nosso escopo, vamos adicionar apenas os seguidores dos nossos usuários que possuem interesse no repositório. Isso facilita nosso trabalho, pois lembramos que a api do github só libera 5000 requisições por hora, e iremos precisar mais adiante.

In [22]:
# percorrendo stargazers
for i, stargazer in enumerate(stargazers):
  try: 
    for follower in stargazer.get_followers():
      # se estiver no grafo, adicione uma edge com o perfil que ele segue
      if follower.login + '(user)' in graph:
        graph.add_edge(follower.login + '(user)', stargazer.login + '(user)', type="follows")
  except Exception as e:
    print("Ocorreu um erro buscando seguidores para ", stargazer.login, "Pulando...")
    
  print("Processados", i+1, "stargazers. Números de nodes/edges", graph.number_of_nodes(), graph.number_of_edges())
  print("Requisições restantes:", client.rate_limiting)

Processados 1 stargazers. Números de nodes/edges 1183 1286
Requisições restantes: (4943, 5000)
Processados 2 stargazers. Números de nodes/edges 1183 1286
Requisições restantes: (4942, 5000)
Processados 3 stargazers. Números de nodes/edges 1183 1286
Requisições restantes: (4941, 5000)
Processados 4 stargazers. Números de nodes/edges 1183 1286
Requisições restantes: (4939, 5000)
Processados 5 stargazers. Números de nodes/edges 1183 1286
Requisições restantes: (4938, 5000)
Processados 6 stargazers. Números de nodes/edges 1183 1286
Requisições restantes: (4937, 5000)
Processados 7 stargazers. Números de nodes/edges 1183 1286
Requisições restantes: (4936, 5000)
Processados 8 stargazers. Números de nodes/edges 1183 1286
Requisições restantes: (4932, 5000)
Processados 9 stargazers. Números de nodes/edges 1183 1286
Requisições restantes: (4927, 5000)
Processados 10 stargazers. Números de nodes/edges 1183 1286
Requisições restantes: (4926, 5000)
Processados 11 stargazers. Números de nodes/edges

### 2.4 Visualizando grafo
Agora podemos visualizar nosso grafo. 

Antes vamos fazer um backup e trabalhar com um grafo cópia para não alterar o original. Vamos salvar o arquivo com a extensão .graphml para podermos utilizar um outro programa para gerar a visualização, o Gephi. Por hora, vamos salvar o arquivo e remover o no de repositório.


In [0]:
# exportando grafo original
nx.write_graphml(graph, "original.graphml")

# cópia do grafo original
new_graph = graph.copy()

# removendo nosso repositório do grafo
new_graph.remove_node('Mining-the-Social-Web(repo)')

In [49]:
# outra cópia para retirar os nós sem arestas
clean_graph = new_graph.copy()

# remove os nós sem arestas
for node in new_graph.nodes:
  counter = 0
  for edge in new_graph.edges:
    if(node == edge[0] or node == edge[1]):
      counter = counter + 1
  
  if(counter == 0):
    clean_graph.remove_node(node)
    
print(nx.info(clean_graph))

Name: Interest graph of a repo
Type: DiGraph
Number of nodes: 722
Number of edges: 1643
Average in degree:   2.2756
Average out degree:   2.2756


In [0]:
# novo grafo
nx.write_graphml(clean_graph, "final.graphml")

Vamos ver como está nosso grafo. Antes de gerar a visualização, podemos remover nosso repositório do grafo, por ser um outlier (todos se conectam com ele). Após isso podemos fazer algumas análises:
- Qual usuário possui mais seguidores nessa rede,
- Qual possui menos
- Quem segue mais
- Quem segue menos

In [50]:
from operator import itemgetter

# total de seguidores
followers = [edge for edge in clean_graph.edges(data=True) if edge[2]['type'] == 'follows']
print("Total de seguidores:", len(followers))

# total de seguidores do dono do repositório
ptwobruseell_followers = [edge for edge in clean_graph.edges(data=True) if edge[2]['type'] == 'follows' and edge[1] == "ptwobrussell(user)"]
print("Total de seguidores de ptwobruseell:", len(ptwobruseell_followers))

# a user who follows many but not followed back by many
print(len(graph.out_edges('mcanthony(user)')))
print(len(graph.in_edges('mcanthony(user)')))

# a user who is followed by many but does not follow back
print(len(graph.out_edges('angusshire(user)')))
print(len(graph.in_edges('angusshire(user)')))

# 10 usuários que possuem mais seguidores
top_10 = sorted([n for n in clean_graph.degree()], key=itemgetter(1), reverse=True)[:10]
print(top_10)

Total de seguidores: 1643
Total de seguidores de ptwobruseell: 129
32
4
495
24
[('angusshire(user)', 518), ('kenneth-reitz(user)', 174), ('ptwobrussell(user)', 129), ('VagrantStory(user)', 107), ('rohithadassanayake(user)', 70), ('trietptm(user)', 70), ('daimajia(user)', 42), ('mcanthony(user)', 35), ('JT5D(user)', 33), ('hammer(user)', 28)]
